# Bosonic Mixture with Non-Local Interaction

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

using FFTW, Interpolations

using UnPack

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

### Hamiltonian

\begin{align}
    H &=  h \sum_{i=1, 2} \left(a^\dagger_{i} a^{\phantom{\dagger}}_{i} - b^\dagger_{i} b^{\phantom{\dagger}}_{i} \right) + J \left( a^\dagger_{1}b^{\phantom{\dagger}}_{1} b^\dagger_{2} a^{\phantom{\dagger}}_{2} + \mathrm{h.c.}\right) 
\end{align}

In [ ]:
# Container for problem data
struct ProblemData
    GL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    GG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    L::Int64
    H::Matrix{ComplexF64}
    J::Float64
  
    # Initialize problem
    function ProblemData(GL0::Matrix{ComplexF64}, L::Int64, H::Matrix{ComplexF64}, J::Float64)
        @assert H == H' "Non-hermitian Hamiltonian"

        data = new(
          GreenFunction(reshape(GL0, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(reshape(GL0 - 1.0im * I, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          L,
          H,
          J
        )

        # Initialize self-energies
        self_energies!(data, 1, 1)

        return data
    end
end

# Self-energy
function self_energies!(data, t1, t2)
    @unpack GL, GG, ΣL, ΣG, L, H, J = data

    if (n = size(GL, 3)) > size(ΣL, 3)
        resize!(ΣL, n)
        resize!(ΣG, n)
    end
  
    NNs = [1, 0] # nearest neighbours
    N_mu = mu -> NNs[mu % L + 1] + mu - mu % L
    idxs = [[(mu + L) % 2L, N_mu(mu), (N_mu(mu) + L) % 2L] .+ 1 for mu in 0:2L-1]
    idxs = hcat(idxs...)
    
#     ΣL[t1, t2] = -J^2 * GL[t1, t2][idxs[1, :], idxs[1, :]] .* GL[t1, t2][idxs[2, :], idxs[2, :]] .* GG[t2, t1][idxs[3, :], idxs[3, :]] |> (diagm ∘ diag)
#     ΣG[t1, t2] = -J^2 * GG[t1, t2][idxs[1, :], idxs[1, :]] .* GG[t1, t2][idxs[2, :], idxs[2, :]] .* GL[t2, t1][idxs[3, :], idxs[3, :]] |> (diagm ∘ diag)
    
    ΣL[t1, t2] = -J^2 * diag(GL[t1, t2])[idxs[1, :]] .* diag(GL[t1, t2])[idxs[2, :]] .* diag(GG[t2, t1])[idxs[3, :]] |> diagm
    ΣG[t1, t2] = -J^2 * diag(GG[t1, t2])[idxs[1, :]] .* diag(GG[t1, t2])[idxs[2, :]] .* diag(GL[t2, t1])[idxs[3, :]] |> diagm
end

In [ ]:
    NNs = [1, 0] # nearest neighbours
    N_mu = mu -> NNs[mu % L + 1] + mu - mu % L
    idxs = [[(mu + L) % 2L, N_mu(mu), (N_mu(mu) + L) % 2L] .+ 1 for mu in 0:2L-1]

In [ ]:
hcat(idxs...)

In [ ]:
# Integration boundaries
tspan = (0.0, 5.0)

# Problem data
data = begin

    # Parameters
    L = 2
    h = 5.0
    H = ComplexF64[-h 0 0 0; 0 -h 0 0; 0 0 h 0; 0 0 0 h];
    J = 1.

    # Initial condition
    delta_n = 0.1
#     GL0 = -1.0im * [0.5 + delta_n 0 0 0; 0 0.5 0 0; 0 0 0.5 - delta_n 0; 0 0 0 0.5]
    GL0 = -1.0im .* 2e-1 .* [2 0 0 0; 0 1 0 0; 0 0 2 0; 0 0 0 0.5]

    ProblemData(GL0, L, H, J)
end;

In [ ]:
function integrate(x::AbstractVector, y::AbstractVector)
    if isone(length(x))
        return zero(first(y))
    end

    @inbounds retval = (x[2] - x[1]) * (y[1] + y[2])
    @inbounds @fastmath @simd for i in 2:(length(y) - 1)
        retval += (x[i+1] - x[i]) * (y[i] + y[i+1])
    end
    return 1//2 * retval
end;

# Vertical rhs
function fv!(out, data, ts, t1, t2)
  @unpack GL, GG, ΣL, ΣG, L, H = data
  
  # real-time collision integral
  ∫dt(i, j, A, B) = sign(j-i) * integrate(ts[min(i, j):max(i, j)], [A[t1, t] * B[t, t2] for t=min(i, j):max(i, j)])

  out[1] = -1.0im * (H * GL[t1, t2] + ∫dt(1, t1, ΣG, GL) - ∫dt(1, t1, ΣL, GL) + ∫dt(1, t2, ΣL, GL) - ∫dt(1, t2, ΣL, GG))
  out[2] = -1.0im * (H * GG[t1, t2] + ∫dt(1, t1, ΣG, GG) - ∫dt(1, t1, ΣL, GG) + ∫dt(1, t2, ΣG, GL) - ∫dt(1, t2, ΣG, GG))
end

# Diagonal rhs
function fd!(out, data, ts, t1, t2)
  fv!(out, data, ts, t1, t2)
  out .-= adjoint.(out)
end

# Integration
sol = kbsolve!(
  (out, x...) -> fv!(out, data, x...), 
  (out, x...) -> (println(" t: $(x[1][x[2]])"); fd!(out, data, x...)), 
  [data.GL, data.GG], 
  tspan; 
  callback = (_, x...) -> self_energies!(data, x...), 
  atol=1e-10, 
  rtol=1e-8);

In [ ]:
sol.t |> size

### Integration with higher-order methods?!

In [ ]:
# Vertical rhs
function fv!(out, data, v1, v2, ts, t1, t2)
  @unpack GL, GG, ΣL, ΣG, L, H = data

  out[1] = -1.0im * (H * GL[t1, t2] + v1[1] + v2[1])
  out[2] = -1.0im * (H * GG[t1, t2] + v1[2] + v2[2])
end

function kv1!(out, data, ts, t1, t2, τ)
    @unpack GL, GG, ΣL, ΣG, L, H = data
    
    out[1] = ΣG[t1, τ] .* GL[τ, t2] - ΣL[t1, τ] .* GL[τ, t2]
    out[2] = ΣG[t1, τ] .* GG[τ, t2] - ΣL[t1, τ] .* GG[τ, t2]                      
end

function kv2!(out, data, ts, t1, t2, τ)
    @unpack GL, GG, ΣL, ΣG, L, H = data
    
    out[1] = ΣL[t1, τ] .* GL[τ, t2] - ΣL[t1, τ] .* GG[τ, t2]
    out[2] = ΣG[t1, τ] .* GL[τ, t2] - ΣG[t1, τ] .* GG[τ, t2]   
end


# Diagonal rhs
function fd!(out, data, v1, v2, ts, t1, t2)
  @unpack GL, GG, ΣL, ΣG, L, H = data
    
  fv!(out, data, v1, v2, ts, t1, t2)
  out .-= adjoint.(out)
end

function kd1!(out, data, ts, t1, t2, τ)
    kv1!(out, data, ts, t1, t2, τ)
end

function kd2!(out, data, ts, t1, t2, τ)
    kv2!(out, data, ts, t1, t2, τ)
end

# Integration
sol = kbsolve!(
  (out, x...) -> fv!(out, data, x...), 
  (out, x...) -> (println(" t: $(x[3][x[4]])"); fd!(out, data, x...)), 
  [data.GL, data.GG], 
  tspan;
  kv1! = (out, x...) -> kv1!(out, data, x...),
  kv2! = (out, x...) -> kv2!(out, data, x...),
  kd1! = (out, x...) -> kv1!(out, data, x...),
  kd2! = (out, x...) -> kv2!(out, data, x...),
  callback = (_, x...) -> self_energies!(data, x...), 
  atol=1e-10, 
  rtol=1e-5);

In [ ]:
sol.t |> size

## QuTiP benchmark

In [ ]:
using PyCall
qt = pyimport("qutip")

In [ ]:
# time parameters
times = range(first(sol.t), stop=last(sol.t), length=128+1) # range(tspan[1]; stop=tspan[2], length=length(state.t));
n = length(times) - 1

### Hamiltonian

In [ ]:
n_max = 2; # Fock-space truncation

# initial state
psi0 = qt.tensor([(sqrt(1 - 1.0im * data.GL[1, 1][k, k]) * qt.basis(n_max + 1, 0) 
            + sqrt(1.0im * data.GL[1, 1][k, k]) * qt.basis(n_max + 1, 1)).unit() for k in 1:2*L]);

# operators
ids = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)]
ops = [deepcopy(ids) for _ in 1:4]

for (i, op) in enumerate(ops)
    op[i] = qt.destroy(n_max + 1)
end

ops = qt.tensor.(ops)
b1, b2, a1, a2 = ops;

In [ ]:
# make diagonal density matrix (i.e. dropping coherences)
rho0 = psi0 * psi0.dag();
for j in 1:rho0.shape[2] , i in 1:rho0.shape[1]
    i != j ? rho0.data[i, j] = 0.0 : continue
end

In [ ]:
H  = -h * b1.dag() * b1
H += -h * b2.dag() * b2
H += +h * a1.dag() * a1
H += +h * a2.dag() * a2
H += J * (a1.dag() * b1 * b2.dag() * a2 + b1.dag() * a1 * a2.dag() * b2)

# observables
obs = [b1.dag() * b1, b2.dag() * b2, a1.dag() * a1, a2.dag() * a2,
       b1.dag() * a1, b2.dag() * a2, b2.dag() * a1, a2.dag() * b1,
       b1 * b1.dag()];

### Simulation

In [ ]:
# quickly solve once for observables
# me = qt.mesolve(H, psi0, times, [], obs)
me = qt.mesolve(H, rho0, times, [], obs)

# solve for the time-dependent density matrix
# t_sols = qt.mesolve(H, psi0, times); # t_sols.states returns state vectors
t_sols = qt.mesolve(H, rho0, times); # t_sols.states returns density matrices

#### Two times

In [ ]:
# -i<X(t2)Y(t1)>
function two_time_average(X, Y, rho_t1)
    X_t2_Y_t1 = zeros(ComplexF64, n + 1, n + 1)
    for k in 1:(n + 1)
        Y_rho_t1 = qt.mesolve(H, Y * rho_t1.states[k], times).states
        for l in 1:(n + 1)
            X_t2_Y_t1[k, l] = -1.0im * (X * Y_rho_t1[l]).tr()    
        end
    end
    
    unskewed_X_t2_Y_t1 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
    for (k, x) in enumerate([X_t2_Y_t1[k, :] for k in 1:(n + 1)])
        for (l, y) in enumerate(x)
            ind = k + l - 1
            unskewed_X_t2_Y_t1[k, ind] = y 
        end
    end    
    return unskewed_X_t2_Y_t1[:, 1:n+1]
end;

In [ ]:
create_destroy = [zeros(ComplexF64, n + 1, n + 1) for _ in 1:1]#2L]
destroy_create = [zeros(ComplexF64, n + 1, n + 1) for _ in 1:1]#2L];

In [ ]:
destroyers = [b1, b2, a1, a2]
for k in 1:1#2L
    create_destroy[k] = two_time_average(destroyers[k].dag(), destroyers[k], t_sols)
    destroy_create[k] = two_time_average(destroyers[k], destroyers[k].dag(), t_sols)
    # flip real part when creation operator is to the right 
    # (equivalent to flipping the sign of tau)
    destroy_create[k] = -conj(destroy_create[k]) #-1.0 * real(destroy_create[k]) + 1.0im * imag(destroy_create[k])
end

In [ ]:
unskewed_b1_dag_b1 = create_destroy[1]
unskewed_b1_b1_dag = destroy_create[1]
# unskewed_b1_dag_b1_minus_b1_b1_dag = (unskewed_b1_dag_b1 .- unskewed_b1_b1_dag) #unskewed_b1_dag_b1_minus_b1_b1_dag[:, 1:n+1];

In [ ]:
full_square = A -> (A - adjoint(A) - (A |> diag |> diagm)) 

# unskewed_b1_dag_b1 = full_square(unskewed_b1_dag_b1)
# unskewed_b1_b1_dag = full_square(unskewed_b1_b1_dag)
# unskewed_b1_dag_b1_minus_b1_b1_dag = full_square(unskewed_b1_dag_b1_minus_b1_b1_dag);

In [ ]:
figure(figsize=(6, 2))
subplot(121)
# imshow(real(b1_dag_b1_minus_b1_b1_dag), cmap="plasma")
imshow(real(full_square(unskewed_b1_dag_b1) |> transpose), cmap="plasma")

subplot(122)
imshow(real(full_square(unskewed_b1_dag_b1)), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
T = sol.t[end]
t_scale = (J == 0. ? 1. : J)
ω_scale = (J == 0. ? 1. : 1/J);

### Equal-time

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
idx = 1
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls="-", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C0")
ax.plot(times, me.expect[idx], "--", c="C0", lw=3, alpha=0.5)
idx = 2
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls=":", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C1")
ax.plot(times, me.expect[idx], "-.", c="C1", lw=3, alpha=0.5)
ax.set_xlabel("\$Jt\$") 
ylabel("\$-\\mathrm{Im}\\; \\mathcal{A}^<_{i,\\, i}(t, t)\$", labelpad=10)
ax.set_xticks([0, 2.5, 5])
ax.set_xlim(0, sol.t[end]) 
ax.set_ylim(0, 0.5)
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
idx = 3
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls="-", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C2")
ax.plot(times, me.expect[idx], "--", c="C2", lw=3, alpha=0.5)
idx = 4
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls=":", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C3")
ax.plot(times, me.expect[idx], "-.", c="C3", lw=3, alpha=0.5)
ax.set_xlabel("\$Jt\$") 
ylabel("\$-\\mathrm{Im}\\; \\mathcal{B}^<_{i,\\, i}(t, t)\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.set_xticks([0, 2.5, 5])
ax.set_yticklabels([])
ax.set_xlim(0, sol.t[end]) 
ax.set_ylim(0, 0.5)
# ax.ticklabel_format(style="sci", axis="y", scilimits=(0, 0))
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)
tight_layout(pad=0.25, w_pad=1, h_pad=0)
savefig("interacting_bosons_example_1.pdf")

### Relative-time

In [ ]:
idx = 1;

In [ ]:
ρ_11_kb = interpolate((sol.t, sol.t), view(data.GL.data .- data.GG.data, idx, idx, :, : ), Gridded(Linear()));
ρ_11_qt = interpolate((times, times), view(full_square(create_destroy[idx] .- destroy_create[idx]), :, : ), Gridded(Linear()));
# ρ_11_qt = full_square(create_destroy[idx] .- destroy_create[idx]); 

In [ ]:
# ρ_11_kb = interpolate((sol.t, sol.t), view(data.GG.data, idx, idx, :, : ), Gridded(Linear()));
# ρ_11_qt = full_square(destroy_create[idx]);

In [ ]:
new_times = range(first(sol.t), stop=last(sol.t), length=2048);

In [ ]:
ρ_11_kb_wigner, (taus, ts) = wigner_transform([ρ_11_kb(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=false);
ρ_11_kb_wigner = ρ_11_kb_wigner

# ρ_11_qt_wigner, (taus_qt, ts_qt) = wigner_transform([ρ_11_qt[t1, t2] for t1 in 1:n+1, t2 in 1:n+1]; ts=times, fourier=false);
ρ_11_qt_wigner, (taus_qt, ts_qt) = wigner_transform([ρ_11_qt(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=false);
ρ_11_qt_wigner = ρ_11_qt_wigner;

ρ_11_kb_FFT, (ωs, ts) = wigner_transform([ρ_11_kb(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=true);
# ρ_11_kb_FFT = -ρ_11_kb_FFT

# ρ_11_qt_FFT, (ωs_qt, ts_qt) = wigner_transform([ρ_11_qt[t1, t2] for t1 in 1:n+1, t2 in 1:n+1]; ts=times, fourier=true);
ρ_11_qt_FFT, (ωs_qt, ts_qt) = wigner_transform([ρ_11_qt(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=true);
# ρ_11_qt_FFT = -ρ_11_qt_FFT;

## Testing

In [ ]:
# plot_kb = ρ_11_kb_FFT[:, center] |> imag
# plot_kb = plot_kb ./ maximum(plot_kb)

# plot_qt = ρ_11_qt_FFT[:, center] |> imag
# plot_qt = plot_qt ./ maximum(plot_qt)

In [ ]:
xpad = 8
ypad = 5

center = floor(length(new_times) / 2) |> Int

figure(figsize=(7, 3))


ax = subplot(121)
# plot(ω_scale * ωs, plot_kb, "-", c="C0", lw=1.5)
# plot(ω_scale * ωs_qt, plot_qt, ls="--", c="C0", lw=2.5, alpha=0.5)
plot(t_scale * taus, ρ_11_kb_wigner[:, center] |> imag, ls="-", c="C0", lw=1.5)
plot(t_scale * taus_qt, ρ_11_qt_wigner[:, center] |> imag, ls="--", c="C0", lw=2.5, alpha=0.5)

# ax.set_xlim(10 .* (-1, 0))
# ax.set_ylim(0, 0.11)
# ax.set_xticks([x for x in -10:5:10])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
# ax.set_xlabel("\$\\omega/J\$")
ax.set_xlabel(L"J \tau")
ax.set_ylabel(L"\mathrm{Re}\,A_{\mathcal{A}_{1,\, 1}}(T, \tau)_W")
ax.set_xlim(-t_scale * T, t_scale * T)
ax.set_ylim(-1.0, 1.0)
# ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
plot(t_scale * taus, -ρ_11_kb_wigner[:, center] |> real, ls="-", c="C0", lw=1.5)
plot(t_scale * taus_qt, -ρ_11_qt_wigner[:, center] |> real, ls="--", c="C0", lw=2.5, alpha=0.5)
ax.set_xlabel(L"J \tau")
ax.set_xlim(-t_scale * T, t_scale * T)
ax.set_ylim(-1.0, 1.0)
# ax.set_xticks(t_scale .* [-T/2, -T/4, 0, T/4, T/2])
ax.set_yticklabels([])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel(L"\mathrm{Im}\,A_{\mathcal{A}_{1,\, 1}}(T, \tau)_W", labelpad=16)
ax.yaxis.set_label_position("right")

tight_layout(pad=0.1, w_pad=1, h_pad=0)

savefig("interacting_bosons_example_2.pdf")

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * taus, ρ_11_kb_wigner[:, Int(floor(n/2))] |> real, ls="-", c="C0", lw=1.5)
plot(t_scale * taus, ρ_11_qt_wigner[:, Int(floor(n/2))] |> real, ls="--", c="C0", lw=2.5)
plot(t_scale * taus, ρ_11_kb_wigner[:, Int(floor(n/2))] |> imag, ls="-", c="C3", lw=1.5)
plot(t_scale * taus, ρ_11_qt_wigner[:, Int(floor(n/2))] |> imag, ls="--", c="C3", lw=2.5)
ax.set_xlabel("\$J \\tau\$")
# ax.set_xlim(-t_scale * T, t_scale * T)
# ax.set_ylim(-1.0, 1.0)
# ax.set_xticks(t_scale .* [-T/2, -T/4, 0, T/4, T/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel(L"A_{0,\, 0}(\tau, T)")

ax = subplot(122)
plot(ω_scale * ωs, ρ_11_kb_FFT[:, Int(floor(n/2))] |> real, ls="-", c="C0", lw=1.5)
plot(ω_scale * ωs, ρ_11_qt_FFT[:, Int(floor(n/2))] |> real, ls="--", c="C0", lw=2.5)
plot(ω_scale * ωs, ρ_11_kb_FFT[:, Int(floor(n/2))] |> imag, ls="-", c="C3", lw=1.5)
plot(ω_scale * ωs, ρ_11_qt_FFT[:, Int(floor(n/2))] |> imag, ls="--", c="C3", lw=2.5)
ax.set_xlabel("\$\\omega/J\$")
ax.set_xlim(10 .* (-1, 1))
# ax.set_ylim(0, 6.0)
ax.set_xticks([x for x in -10:5:10])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$A_{0,\\, 0}(\\omega, T)\$", labelpad=16)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

tight_layout(pad=0.1, w_pad=0.5, h_pad=0)

# savefig("interacting_bosons_example_tau_omega.pdf")

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
cmap = "gist_heat"
Y, X = meshgrid(sol.t[1:end], sol.t[1:end]);

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale .* times, full_square(unskewed_b1_dag_b1) |> diag |> (-) |> imag, "-C0", lw=1.5)
plot(t_scale .* times, (unskewed_b1_b1_dag - unskewed_b1_dag_b1) |> diag |> (-) |> imag, ":C0", lw=1.5)
plot(t_scale .* times, me.expect[1], "--C3", lw=1.5)
# plot(t_scale .* times, me.expect[end] - me.expect[1], ":C7", lw=1.5)

ax = subplot(122)
plot(t_scale .* times, full_square(unskewed_b1_b1_dag) |> diag |> (-) |> imag, "-C0", lw=1.5)

plot(t_scale .* times, me.expect[1], "--C3", lw=1.5)
plot(t_scale .* times, me.expect[end], "--C3", lw=1.5)

tight_layout()

In [ ]:
plot(sol.t, sum([data.GL.data[k, k ,:,:] - data.GG.data[k, k ,:,:] |> ((-) ∘ imag ∘ diag) for k in 1:4]))

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
vmin = -0.5 # minimum(ρ_11)
vmax = 1.0 # maximum(ρ_11)
# ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, data.GG.data[1, 1, :, :] |> imag, cmap=cmap, rasterized=true)#, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])


ax = subplot(122)
vmin = -0.5 # minimum(ρ_11)
vmax = 1.0 # maximum(ρ_11)
# ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, (data.GL.data[1, 1, :, :] - data.GG.data[1, 1, :, :]) |> imag, cmap=cmap, rasterized=true)#, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
colorbar(mappable=heatmap)
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout()

In [ ]:
test_kb = interpolate((sol.t, sol.t), view(-data.GL.data[3, 3, :, :] .* data.GG.data[1, 1, :, :], :, :), Gridded(Linear()));
# test_kb = interpolate((sol.t, sol.t), view(-data.GL.data, 2, 2, :, :), Gridded(Linear()));
test_kb_wigner, (test_taus, test_ts) = wigner_transform([test_kb(t1, t2) for t1 in times, t2 in times]; ts=times, fourier=false);
test_kb_wigner = test_kb_wigner |> imag;

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
vmin = -0.5 # minimum(ρ_11)
vmax = 1.0 # maximum(ρ_11)
# ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, test_kb(times, times) |> real, cmap=cmap, rasterized=true)#, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
colorbar(mappable=heatmap)